# 9.3 흉부 엑스선을 기반으로 폐렴 진단 베이스라인 모델
[흉부 엑스선 기반 폐렴 진단 데이터 세트 링크](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia)

In [1]:
# 데이터 경로
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'

# 훈련, 검증, 테스트 데이터 경로 설정
train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

## 9.3.1 시드 값 고정 및 GPU 장비 설정

### 시드 값 고정

In [2]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드 값 고정
seed = 10

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 9.3.2 데이터 준비하기

### 데이터 증강을 위한 이미지 변환기 정의

In [4]:
from torchvision import transforms

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
                          transforms.Resize((224, 224)), # 이미지 크기 조정 
                          transforms.CenterCrop(180), # 중앙 이미지 확대
                          transforms.RandomHorizontalFlip(), # 좌우 대칭
                          transforms.RandomVerticalFlip(), # 상하 대칭
                          transforms.RandomRotation(20), # 이미지 회전
                          transforms.ToTensor(), # 텐서 객체로 변환
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))]) # 정규화

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
                          transforms.Resize((224, 224)),
                          transforms.ToTensor(),
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))])

### 데이터 세트 및 데이터 로더 생성

In [5]:
from torchvision.datasets import ImageFolder

# 훈련 데이터 세트
datasets_train = ImageFolder(root=train_path, transform=transform_train)
# 검증 데이터 세트
datasets_valid = ImageFolder(root=valid_path, transform=transform_test) 

In [6]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 제너레이터 시드 값 고정
g = torch.Generator()
g.manual_seed(0)

In [7]:
from torch.utils.data import DataLoader

batch_size = 16

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g)

## 9.3.3 모델 생성 및 훈련

### 모델 생성

In [8]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=c39c022a24dc79c3289c2c1f4bc70d75354016f9979c892bfa7ad6ce36066159
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [9]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=2) 

model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [10]:
print('모델 파라미터 개수 :', sum(param.numel() for param in model.parameters()))

모델 파라미터 개수 : 63792082


### 손실 함수, 옵티마이저

In [11]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.001)

### 모델 훈련 및 성능 검증

In [13]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score # 재현율 계산 함수
from sklearn.metrics import f1_score # F1 점수 계산 함수

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=5, save_file='model_state_dict.pth'):
    
    valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) ---①

    # 총 에폭만큼 훈련
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        model.train() # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 
        for images, labels in loader_train:
            # 이미지, 레이블(타깃 값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            loss.backward() # 역전파 수행
            optimizer.step() # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 ---②
                scheduler.step() 
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')

        model.eval() # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        all_preds = [] # 예측값 저장용 리스트 초기화
        all_true = [] # 실제값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 ---③
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
    
                all_preds.extend(preds)
                all_true.extend(true)
                
            # 정확도, 재현율, F1 점수 ---④
            val_accuracy = accuracy_score(all_true, all_preds)
            val_recall = recall_score(all_true, all_preds)
            val_f1_score = f1_score(all_true, all_preds)

            # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
            print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
            print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')

            # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 ---⑤
            if epoch_valid_loss <= valid_loss_min: 
                print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
                # 모델 가중치를 파일로 저장 ---⑥
                torch.save(model.state_dict(), save_file) 
                valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 ---⑦
    return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

In [14]:
# 모델 훈련
model_state_dict = train(model=model,
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion, 
                         optimizer=optimizer)

에폭 [1/5] 
-----------------------------
	훈련 데이터 손실값 : 0.2011
	검증 데이터 손실값 : 2.1317
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
	### 검증 데이터 손실값 감소 (inf --> 2.1317). 모델 저장
에폭 [2/5] 
-----------------------------
	훈련 데이터 손실값 : 0.1084
	검증 데이터 손실값 : 1.6576
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
	### 검증 데이터 손실값 감소 (2.1317 --> 1.6576). 모델 저장
에폭 [3/5] 
-----------------------------
	훈련 데이터 손실값 : 0.0960
	검증 데이터 손실값 : 1.0366
	정확도 : 0.5625 / 재현율 : 1.0000 / F1 점수 : 0.6957
	### 검증 데이터 손실값 감소 (1.6576 --> 1.0366). 모델 저장
에폭 [4/5] 
-----------------------------
	훈련 데이터 손실값 : 0.0765
	검증 데이터 손실값 : 0.7975
	정확도 : 0.5625 / 재현율 : 1.0000 / F1 점수 : 0.6957
	### 검증 데이터 손실값 감소 (1.0366 --> 0.7975). 모델 저장
에폭 [5/5] 
-----------------------------
	훈련 데이터 손실값 : 0.0626
	검증 데이터 손실값 : 1.1829
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667


In [15]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

## 9.3.4 예측 및 평가 결과

In [16]:
datasets_test = ImageFolder(root=test_path, transform=transform_test)

loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g)

### 예측

In [17]:
def test(model, loader_test, return_true=False):
    model.eval() # 모델을 평가 상태로 설정
    all_preds = [] # 예측값 저장용 리스트 초기화
    all_true = [] # 실제값 저장용 리스트 초기화

    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값
            true = labels.cpu().numpy() # 실제값 

            all_preds.extend(preds)
            all_true.extend(true)

    if return_true:
        return all_true, all_preds
    else:
        return all_preds

In [18]:
all_true, all_preds = test(model=model, 
                           loader_test=loader_test, 
                           return_true=True)

### 평가 결과

In [19]:
print('#'*5, '최종 예측 결과 평가 점수', '#'*5)
print(f'정확도 : {accuracy_score(all_true, all_preds):.4f}')
print(f'재현율 : {recall_score(all_true, all_preds):.4f}')
print(f'F1 점수 : {f1_score(all_true, all_preds):.4f}')

##### 최종 예측 결과 평가 점수 #####
정확도 : 0.7628
재현율 : 0.9923
F1 점수 : 0.8395
